In [1]:
import gpytorch
import torch
import numpy as np

In [2]:
from demandSimulate import demandSimulate

xs = demandSimulate(0.2,0,1,3,n_sim = 60000,maturity= 1,P0 = 0 )

In [4]:
import LBFGS

In [3]:
responses = np.sum(xs,axis = 1)


xs = xs[:,1:]

In [4]:
train_x = torch.Tensor(xs)
train_y = torch.Tensor(responses)

In [5]:
train_x.shape

torch.Size([60000, 3])

In [6]:
output_device = torch.device("mps")

In [7]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        base_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

        self.covar_module = gpytorch.kernels.MultiDeviceKernel(
            base_covar_module,
            output_device=output_device)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [50]:
def train(train_x,
          train_y,
          output_device,
          preconditioner_size,
          n_training_iter,
):
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
    model = ExactGPModel(train_x, train_y, likelihood).to(output_device)
    model.train()
    likelihood.train()

    optimizer = FullBatchLBFGS(model.parameters(), lr=0.1)
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


    with gpytorch.settings.max_preconditioner_size(preconditioner_size):

        def closure():
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            return loss

        loss = closure()
        loss.backward()

        for i in range(n_training_iter):
            options = {'closure': closure, 'current_loss': loss, 'max_ls': 10}
            loss, _, _, _, _, _, _, fail = optimizer.step(options)

            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, n_training_iter, loss.item(),
                model.covar_module.module.base_kernel.lengthscale.item(),
                model.likelihood.noise.item()
            ))

            if fail:
                print('Convergence reached!')
                break

    print(f"Finished training on {train_x.size(0)} data points using {1} GPUs.")
    return model, likelihood

In [51]:
model, likelihood = train(train_x, train_y,
                           output_device=output_device,
                          preconditioner_size=100,
                          n_training_iter=20)

TypeError: MultiDeviceKernel.__init__() missing 1 required positional argument: 'device_ids'